In [1]:
!pip install datasets
!pip install trl

In [2]:
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer  # trl has a lot of reinforcement l approaches
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

load data & model

In [3]:
dataset = load_dataset("kri6521/youtube_titles_dpo")

model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # set pad token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
def format_chat_prompt(user_input, system_message="You are a helpful assistant."):
    """
    Formats user input into the chat template format with <|im_start|> and <|im_end|> tags.

    Args:
        user_input (str): The input text from the user.

    Returns:
        str: Formatted prompt for the model.
    """

    # Format user message
    user_prompt = f"<|im_start|>user\n{user_input}<|im_end|>\n"

    # Start assistant's turn
    assistant_prompt = "<|im_start|>assistant\n"

    # Combine prompts
    formatted_prompt = user_prompt + assistant_prompt

    return formatted_prompt

In [6]:
print(dataset.keys())

dict_keys(['train', 'validation'])


In [7]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = format_chat_prompt(dataset['validation']['prompt'][0][0]['content'])
outputs = generator(prompt, max_length=100, truncation=True,
                    num_return_sequences=1, temperature=0.7)
print(outputs[0]['generated_text'])

Device set to use cpu


<|im_start|>user
Given the YouTube video idea write an engaging title.

**Video Idea**: intro independent component analysis

**Additional Guidance**:
- Title should be between 30 and 75 characters long
- Only return the title idea, nothing else!<|im_end|>
<|im_start|>assistant
"Unlocking Independent Component Analysis: Exploring Your Data!"


In [8]:
ft_model_name = model_name.split('/')[1].replace("Instruct", "DPO")

training_args = DPOConfig(
    output_dir=ft_model_name,
    logging_steps=25,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="epoch",
    eval_strategy="epoch",
    eval_steps=1,
)

device = torch.device('mps')

In [ ]:
trainer = DPOTrainer(
    model=model,
    args=training_args,
    processing_class=tokenizer,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
)
trainer.train()

In [ ]:
# Load the fine-tuned model
ft_model = trainer.model
# Set up text generation pipeline
generator = pipeline("text-generation", model=ft_model, tokenizer=tokenizer, device='mps')

# Example prompt
prompt = format_chat_prompt(dataset['valid']['prompt'][0][0]['content'])

# Generate output
outputs = generator(prompt, max_length=100, truncation=True, num_return_sequences=1, temperature=0.7)

print(outputs[0]['generated_text'])